# __Finance 5350: Computational Financial Modeling__

## __Introduction to the Black-Scholes-Merton Model__

The BSM formula is given by the following (for a call option):

$$
C(S, K, \sigma,r, T, \delta) = S e^{-\delta T} N(d_{1}) - K e^{-r T} N(d_{2})
$$

<br>

and by (for a put option):

$$
P(S, K, \sigma,r, T, \delta) = K e^{-rT}N(-d_{2}) - S e^{-\delta T} N(-d_{1})
$$

<br>

where

$$
\begin{align}
d_{1} &= \frac{\ln{(S/K)} + (r - \delta + \frac{1}{2} \sigma^{2}) T}{\sigma \sqrt{T}} \\
d_{2} &= d_{1} - \sigma \sqrt{T}
\end{align}
$$

<br>

Let's see the call option formula as a pricer function:

In [3]:
import numpy as np
from scipy.stats import norm

In [4]:
def blackScholesCall(spot, strike, vol, rate, expiry, div):
    d1 = (np.log(spot / strike) + (rate - div + 0.5 * vol * vol) * expiry) / (vol * np.sqrt(expiry))
    d2 = d1 - (vol * np.sqrt(expiry))
    callPrc = (spot * np.exp(-div * expiry) * norm.cdf(d1)) - (strike * np.exp(-rate * expiry) * norm.cdf(d2))
    
    return callPrc

In [5]:
## Basic Data (see McDonald Chp 12 page 377 2nd Edition)
spot = 41.0
strike = 40.0
vol = 0.30
rate = 0.08
expiry = 0.25
div = 0.0

callPrc = blackScholesCall(spot, strike, vol, rate, expiry, div)
callPrc

3.3990781872368956

In [6]:
## Call Payoff Function
def callPayoff(spot, strike):
    return np.maximum(spot - strike, 0.0)

## Put Payoff Function
def putPayoff(spot, strike):
    return np.maximum(strike - spot, 0.0)

In [7]:
from scipy.stats import binom

def binomialPricer(S, K, r, v, q, T, n, payoff, verbose = True):
    nodes = n  + 1
    h = T / n
    u = np.exp((r - q) * h + v * np.sqrt(h))
    d = np.exp((r - q) * h - v * np.sqrt(h))
    pstar = (np.exp((r - q) * h) - d) / (u - d)
    
    price = 0.0
    
    for i in range(nodes):
        prob = binom.pmf(i, n, pstar)
        spotT = S * (u ** i) * (d ** (n - i))
        po = payoff(spotT, K) 
        price += po * prob
        if verbose:
            print(f"({spotT:0.4f}, {po:0.4f}, {prob:0.4f})")
        
    price *= np.exp(-r * T)
    
    return price


In [8]:
N = [10, 25, 50, 75, 100, 125, 150, 175, 200, 500]

for i in range(len(N)):
    nsteps = N[i]
    callPrc = binomialPricer(spot, strike, rate, vol, div, expiry, nsteps, callPayoff, verbose=False)
    print(callPrc)

3.4544402138080796
3.408694981783251
3.3917972818360145
3.401464570319705
3.4046182898109367
3.3993570672462536
3.399604531287815
3.4022193238912726
3.3982871838738475
3.399919762643824


## __The Option Greeks__

___Option Greeks___ are formulas that express the change in the option price when an input to the formula changes, holding all other parameters fixed. These are the partial derivatives of the BSM formula with respect to each of its parameters. 

### __Definition of the Greeks__

- __Delta ($\Delta$):__ measures the option price change when the stock price increases by $\$1$.

- __Gamma ($\Gamma$):__ measures the change in $\Delta$ when the stock price increases by $\$1$.

- __Vega:__ measures the change in the option price when there is an increase in volatility of 1 percentage point. (No Greek letter for this one, they just wanted one to signify volatility)

- __Theta ($\Theta$):__ measures the change in the option price when there is a decrease in the time to maturity of 1 day.

- __Rho ($\rho$):__ measures the change in the option price when there is an increase in the interest rate of 1 percentage point (100 basis points)

- __Psi ($\Psi$):__ measures the change in the option price when there is an increase in the continuous dividend yield of 1 percentage point (100 basis points)

### __The Formulas for Option Greeks__


#### __Delta ($\Delta$)__

Delta measures the change in the option price for a $\$1$ change in the stock price:

<br>

$$
\begin{align}
\mbox{Call Delta} &= \frac{\partial C(S, K, \sigma, r, T - t, \delta)}{\partial S} = e^{-\delta (T - t)} N(d_{1}) \\
\mbox{Put Delta} &= \frac{\partial P(S, K, \sigma, r, T - t, \delta)}{\partial S} = e^{-\delta (T - t)} N(-d_{1})
\end{align}
$$

<br>

`Python` code for these are given below:

In [25]:
def bsmCallDelta(spot, strike, vol, rate, tau, div):
    d1 = (np.log(spot / strike) + (rate - div + 0.5 * vol * vol) * tau) / (vol * np.sqrt(tau))
    delta = np.exp(-div * tau) * norm.cdf(d1)
    
    return delta

In [26]:
## McDonald Chapter 13 (page 383 3rd Edition)
spot = 40.0
strike = 40.0
vol = 0.30
rate = 0.08
tau = 91/365
div = 0.0

callDelta = bsmCallDelta(spot, strike, vol, rate, tau, div)
print(f"The Call Delta is: {callDelta : 0.4f}")

The Call Delta is:  0.5824


In [27]:
def bsmPutDelta(spot, strike, vol, rate, tau, div):
    d1 = (np.log(spot / strike) + (rate - div + 0.5 * vol * vol) * tau) / (vol * np.sqrt(tau))
    delta = np.exp(-div * tau) * norm.cdf(-d1)

    return delta

In [28]:
putDelta = bsmPutDelta(spot, strike, vol, rate, tau, div)
print(f"The Put Delta is: {putDelta : 0.4f}")

The Put Delta is:  0.4176


#### __Gamma ($\Gamma$)__

Gamma measures the change in delta when the stock price changes. 

<br>

$$
\begin{align}
\mbox{Call Gamma} &=  \frac{\partial^{2} C(S, K, \sigma, r, T - t, \delta)}{\partial S^{2}} = \frac{e^{-\delta (T - t)} N^{\prime}(d_{1})}{S \sigma \sqrt{T - t}} \\
\mbox{Put Gamma} &= \frac{\partial^{2} P(S, K, \sigma, r, T - t, \delta)}{\partial S^{2}} = \mbox{Call Gamma}
\end{align}
$$

<br>

#### __Theta ($\Theta$)__

Theta measures the change in the option price with respect to calendar time ($t$), holding fixed time to expiration ($T$):

<br>

$$
\begin{align}
\mbox{Call Theta} &= \frac{\partial C(S, K, \sigma, r, T - t, \delta)}{\partial t} = \delta S^{-\delta (T-t)} N(d_{1}) - r K e^{-r(T-t)} N(d_{2}) - \frac{K e^{-r(T-t)} N^{\prime}(d_{2}) \sigma}{2 \sqrt{T -t}} \\
\mbox{Put Theta} &= \frac{\partial P(S, K, \sigma, r, T - t, \delta)}{\partial t} = \mbox{Call Theta} + r K e^{-r(T-t)} - \delta S e^{-\delta (T-t)}
\end{align}
$$


<br>

#### __Vega__

Vega measures the change in the option price when volatility changes. Some writers also use the term _lambda_ or _kappa_ to refer to this measure:

<br>

$$
\begin{align}
\mbox{Call vega} &= \frac{\partial C(S, K, \sigma, r, T - t, \delta)}{\partial \sigma} = S e^{-\delta (T-t)} N^{\prime}(d_{1}) \sqrt{T - t} \\
\mbox{Put vega} &= \frac{\partial P(S, K, \sigma, r, T - t, \delta)}{\partial \sigma} = \mbox{Call vega}
\end{align}
$$

<br>

#### __Rho ($\rho$)__

Rho is the partial derivative of the option price with respect  to the interest rate: 

<br>

$$
\begin{align}
\mbox{Call rho} &= \frac{\partial C(S, K, \sigma, r, T - t, \delta)}{\partial r} = (T - t) K e^{-r (T - t)} N(d_{2}) \\
\mbox{Put rho} &= \frac{\partial P(S, K, \sigma, r, T - t, \delta)}{\partial r} = -(T - t) K e^{-r (T - t)} N(-d_{2})
\end{align}
$$

<br>

_Note:_ These expressions for rho assume a change in $r$ of $1.0$. We are typically interested in evaluating the effect of a change of $0.01$ (100 basis points) or $0.0001$ (1 basis point). To report rho as a change per percentage point in the interest rate, divide this measure by 100. To interpret it as a change per basis point, divide by 10,000.

<br>

#### __Psi ($\Psi$)__

Psi is the partial derivative of the option price with respect to the continuous divident yield:

$$
\begin{align}
\mbox{Call Psi} &= \frac{\partial C(S, K, \sigma, r, T - t, \delta)}{\partial \delta} = -(T - t) S e^{-\delta (T - t)} N(d_{1}) \\
\mbox{Put Psi} &= \frac{\partial P(S, K, \sigma, r, T - t, \delta)}{\partial \delta} = (T - t) S e^{-r (T - t)} N(-d_{1}) 
\end{align}
$$

<br>

_Note:_ To interpret Psi as a price change per percentage point change in the dividend yield, divide by 100. 

<br>

_Note:_ $N^{\prime}(\cdot)$ is the Standard Normal PDF, while $N(\cdot)$ is the Standard Normal CDF.